<a href="https://colab.research.google.com/github/mateuszbarnacki/ML-Proj/blob/main/ThousandMinMax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from enum import Enum
from random import shuffle, randint
from math import floor
'''
Assumptions:  we have only 2 players
'''
'''
9 - 0
10 - 10
J - 2
D - 3
K - 4
A - 11
'''

class CardColor(Enum):
  PIK = 'PIK'
  TREFL = 'TREFL'
  KARO = 'KARO'
  KIER = 'KIER'

class CardValue(Enum):
  NINE = 0
  TEN = 10
  J = 2
  D = 3
  K = 4
  A = 11

class Card:
  def __init__(self, value, color):
    self.value = value
    self.color = color

  def show(self):
    print(self.value, ' ', self.color)

class Pack:
  def __init__(self):
    self.pack = self.createPack()
  
  def generatePack(self):
    pack = []
    for cardColor in CardColor:
      for cardvalue in CardValue:
        pack.append( Card(cardvalue.value, cardColor.value) )
    return pack
  
  def shufflePack(self, pack):
    return shuffle(pack)

  def createPack(self):
    pack = self.generatePack()
    self.shufflePack(pack)
    return pack
  
  def show(self):
    mappedPack = list(map(lambda card: ({
        'color': CardColor(card.color).name, 
        'value': CardValue(card.value).name 
        }),self.pack))
    for card in mappedPack:
      print(card['color'], ' ', card['value'])
  
  def getPack(self):
    return self.pack

class UserCards:
  def __init__(self, cards, dispatchesValues):
    self.cards = cards
    self.dispatchesValues = dispatchesValues
    self.estimatedCardsValue = self.estimateCardValues()

  def getCards(self):
    return self.cards
  
  def getCardsQuantity(self):
    return len(self.cards)

  def showCards(self):
    mappedPack = list(map(lambda card: ({
        'color': CardColor(card.color).name, 
        'value': CardValue(card.value).name 
        }),self.cards))
    for card in mappedPack:
      print(card['color'], ' ', card['value'])
  
  def addCards(self, cards):
    self.cards.extend(cards)
  
  def checkDispatches(self):
    kingsAndQueens = list(filter(lambda card: card.value ==  CardValue.D.value or card.value == CardValue.K.value, self.cards))
    possibleDispatches = {}
    for card in kingsAndQueens:
      if card.color in possibleDispatches:
        possibleDispatches[card.color].append(card.value)
      else:
        possibleDispatches[card.color] = [card.value]

    dispatches = []
    for color in possibleDispatches:
      if len(possibleDispatches[color]) == 2:
        dispatches.append(color)
    
    return dispatches

  def estimateCardValues(self):
    estimatedValue = 0
    dispatches = self.checkDispatches()
    
    for dispatch in dispatches:
      estimatedValue += self.dispatchesValues[dispatch]
    
    for card in self.cards:
      estimatedValue += card.value

    return floor(estimatedValue/10)*10

  def isWorthToTakePartInBidding(self, currentPrice):
    return self.estimatedCardsValue >= currentPrice

  def rejectWorthlessCards(self, quantityToReject):
    '''SHOULD BE SOME EXTRA MIN MAX LOGIC HERE'''
    '''
    HERE SHOULD BE IMPLEMENTED THE MINMAX TO DECIDE WHICH CARDS ARE THE BEST FOR PLAYER
    '''
    '''TEMPORARY'''
    while quantityToReject > 0:
      indexToRemove = randint(0, len(self.cards) - 1)
      self.cards.pop(indexToRemove)
      quantityToReject -= 1

  def handleExtraCards(self, extraCards):
    '''We have to assume that we add 4 cards to player. It is because we should not have cards that are not known in the game.'''
    self.addCards(extraCards)
    self.rejectWorthlessCards(4)

  def getValueToWin(self, minValueToWin):
    newEstimatedCardValues = self.estimateCardValues()

    if newEstimatedCardValues > minValueToWin:
      return newEstimatedCardValues
    return minValueToWin 

  def makeDecision(self, leadingGame, activeDispatch, leadingCard):
    '''MINMAX TO MAKING DECISION WHICH CARD SHOULD BE GIVEN'''
    return self.cards.pop(), False

class Dispatch:
  def __init__(self, dispatchesValues):
    self.color = None
    self.isActive = False
    self.dispatchesValues = dispatchesValues
  
  def setDispatch(self, color):
    self.color = color
    self.isActive = True
  
  def getDispatchColor(self):
    if self.isActive:
      return self.color
    return None
  
  def getDispatchValue(self):
    return self.dispatchesValues[self.color]

class Game:
  def __init__(self):
    self.pack = Pack()
    self.valueToWin = 0
    self.users = []
    self.indexOfUserLeadingGame = -1
    self.usersPoints = [0, 0]
    self.iterations = -1
    self.dispatchesValues = {
        "PIK" : 40,
        "TREFL": 60,
        "KARO": 80,
        "KIER": 100
    }

  def playRound(self):
    self.prepareGame()
    return self.play()

  def play(self):
    currentCardsOnBoard = [None, None]
    activeDispatch = Dispatch(self.dispatchesValues)

    while self.iterations > 0:
      currentCardsOnBoard[self.indexOfUserLeadingGame], isItDispatch = self.users[self.indexOfUserLeadingGame].makeDecision(True, activeDispatch, None)
      currentCardsOnBoard[self.getIndexOfNextUser()], unusedVariable = self.users[self.getIndexOfNextUser()].makeDecision(False, activeDispatch, currentCardsOnBoard[self.indexOfUserLeadingGame])

      self.handleDispatch(isItDispatch, activeDispatch, currentCardsOnBoard)
      self.resolveCards(currentCardsOnBoard, activeDispatch)
      self.iterations -= 1
    return self.usersPoints

  def resolveCards(self, currentCardsOnBoard, activeDispatch):
    dispatchColor = activeDispatch.getDispatchColor()
    cardsValue = currentCardsOnBoard[self.indexOfUserLeadingGame].value + currentCardsOnBoard[self.getIndexOfNextUser()].value
    
    if dispatchColor:
      if currentCardsOnBoard[self.indexOfUserLeadingGame].color == dispatchColor and currentCardsOnBoard[self.getIndexOfNextUser()].color == dispatchColor:
        if currentCardsOnBoard[self.indexOfUserLeadingGame].value < currentCardsOnBoard[self.getIndexOfNextUser()].value:
          self.indexOfUserLeadingGame = self.getIndexOfNextUser()
      
        self.usersPoints[self.indexOfUserLeadingGame] += cardsValue

      else:
        if currentCardsOnBoard[self.indexOfUserLeadingGame].color == dispatchColor:
          self.usersPoints[self.indexOfUserLeadingGame] += cardsValue
        
        elif currentCardsOnBoard[self.getIndexOfNextUser()].color == dispatchColor:
          self.indexOfUserLeadingGame = self.getIndexOfNextUser()
          self.usersPoints[self.indexOfUserLeadingGame] += cardsValue
        
        else:
          if currentCardsOnBoard[self.indexOfUserLeadingGame].color == currentCardsOnBoard[self.getIndexOfNextUser()].color and currentCardsOnBoard[self.indexOfUserLeadingGame].value < currentCardsOnBoard[self.getIndexOfNextUser()].value:
            self.indexOfUserLeadingGame = self.getIndexOfNextUser()

          self.usersPoints[self.indexOfUserLeadingGame] += cardsValue

    else:
      if currentCardsOnBoard[self.indexOfUserLeadingGame].color == currentCardsOnBoard[self.getIndexOfNextUser()].color and currentCardsOnBoard[self.indexOfUserLeadingGame].value < currentCardsOnBoard[self.getIndexOfNextUser()].value:
        self.indexOfUserLeadingGame = self.getIndexOfNextUser()

      self.usersPoints[self.indexOfUserLeadingGame] += cardsValue



  def handleDispatch(self, isItDispatch, activeDispatch, currentCardsOnBoard):
    if isItDispatch:
        activeDispatch.setDispatch(currentCardsOnBoard[self.indexOfUserLeadingGame].color)
        self.usersPoints[self.indexOfUserLeadingGame] += activeDispatch.getDispatchValue()
        isItDispatch = False
    return isItDispatch

  def prepareGame(self):
    #Rozdanie
    users, hiddenCards = self.dealCards()
    
    #Licytacja
    indexOfUserLeadingGame, minValueToWin = self.bidding(users)

    #Zabranie karty przez wygranego licytacje
    users[indexOfUserLeadingGame].handleExtraCards(hiddenCards)

    #Podbicie stawki kart w licytacji
    minValueToWin = users[indexOfUserLeadingGame].getValueToWin(minValueToWin)

    self.valueToWin = minValueToWin
    self.users = users
    self.indexOfUserLeadingGame = indexOfUserLeadingGame
    self.iterations = users[indexOfUserLeadingGame].getCardsQuantity()

  def dealCards(self):
    cards = [[],[]]
    packList = self.pack.getPack()
    hiddenCards = packList[-4:]

    for cardIndex in range(len(packList) - 4):
      cards[cardIndex % 2].append(packList[cardIndex])

    usersCards = [ UserCards(cardsForUser, self.dispatchesValues) for cardsForUser in cards]
    
    return usersCards, hiddenCards

  def bidding(self, users):
    currentPrice = 100
    leadingUserIndex = 0

    while True:
      if users[(leadingUserIndex + 1) % 2].isWorthToTakePartInBidding(currentPrice):
        currentPrice += 10
        leadingUserIndex = (leadingUserIndex + 1) % 2
      else:
        break
    return leadingUserIndex, currentPrice
  
  def getIndexOfNextUser(self):
    if self.indexOfUserLeadingGame == 1:
      return 0
    return 1


usersPoints = Game().playRound()
usersPoints

[64, 37]